## Mapparser.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Your task is to use the iterative parsing to process the map file and
find out not only what tags are there, but also how many, to get the
feeling on how much of which data you can expect to have in the map.
Fill out the count_tags function. It should return a dictionary with the 
tag name as the key and number of times this tag can be encountered in 
the map as value.

Note that your code will be tested with a different data file than the 'example.osm'
"""
import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag in tags.keys():
            tags[elem.tag] +=1
        else:
            tags[elem.tag] = 1
    return tags



def test():

    tags = count_tags('dallas.osm')
    pprint.pprint(tags)
       

if __name__ == "__main__":
    test()

## Tags.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
"""
Your task is to explore the data a bit more.
Before you process the data and add it into your database, you should check the
"k" value for each "<tag>" and see if there are any potential problems.

We have provided you with 3 regular expressions to check for certain patterns
in the tags. As we saw in the quiz earlier, we would like to change the data
model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with
problematic characters.

Please complete the function 'key_type', such that we have a count of each of
four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
    
        key = element.attrib['k']
        if re.search(lower, key):
            keys['lower'] +=1
        elif re.search(lower_colon, key):
            keys['lower_colon'] +=1
        elif re.search(problemchars, key):
            keys['problemchars']+=1
        else:
            keys['other']+=1
        pass
    element.clear()
    return keys    


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



def test():
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertion below will be incorrect then.
    # Note as well that the test function here is only used in the Test Run;
    # when you submit, your code will be checked against a different dataset.
    keys = process_map('dallas.osm')
    pprint.pprint(keys)



if __name__ == "__main__":
    test()

### Users.py

In [ ]:
import xml.etree.cElementTree as ET


def get_user(element):
    return 


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(element.get('uid'))
   
    print(len(users))
    return users



if __name__ == "__main__":
    process_map('dallas.osm')

## User_count.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET



def get_user(element):
    return 


def process_map(filename):
    users = set()
    dup_att = {}
   
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            dup_attrib = element.get('uid')
            if dup_attrib in users:
                dup_att[dup_attrib] +=1
            else:
                users.add(element.get('uid'))
                dup_att[dup_attrib] = 1
        element.clear()
    for k,v in dup_att.items():
        print (k, v)
       
                
   

    return users




if __name__ == "__main__":
    process_map('dallas.osm')

## Audit.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  5 00:44:55 2018

@author: Kris
"""

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

#OSMFILE = "dallas.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


#https://en.wikipedia.org/wiki/Street_suffix
#https://pe.usps.com/text/pub28/28apc_002.htm
#https://en.wikipedia.org/wiki/Texas_State_Highway_Spur_408
expected = ['Alley', 'Annex', 'Arcade', 'Avenue', 'Bayou', 'Beach', 'Bend', 'Bluff', 'Bottom', 'Boulevard', 'Branch', 'Bridge',\
            'Brook', 'Burg', 'Bypass', 'Camp', 'Canyon', 'Cape', 'Causeway', 'Center', 'Circle', 'Cliff', 'Club', 'Common',\
            'Corner', 'Course', 'Court', 'Cove', 'Creek', 'Crescent', 'Crest', 'Crossing', 'Crossroad', 'Curve', 'Dale', \
            'Dam', 'Divide', 'Drive', 'Estate', 'Expressway', 'Extension', 'Fall', 'Ferry', 'Field', 'Flat', 'Ford', \
            'Forest', 'Forge', 'Fork', 'Fort', 'Freeway', 'Garden', 'Gateway', 'Glen', 'Green', 'Grove', 'Harbor', \
            'Haven', 'Heights', 'Highway', 'Hill', 'Hollow', 'Inlet', 'Island', 'Isle', 'Junction', 'Key', 'Knoll', \
            'Lake', 'Land', 'Landing', 'Lane', 'Light', 'Loaf', 'Lock', 'Lodge', 'Loop', 'Mall', 'Manor', 'Meadow', \
            'Mews', 'Mill', 'Mission', 'Motorway', 'Mount', 'Mountain', 'Neck', 'Orchard', 'Oval', 'Overpass', 'Park', \
            'Parkway', 'Pass', 'Passage', 'Path', 'Pike', 'Pine', 'Place', 'Plain', 'Plaza', 'Point', 'Port', 'Prairie', \
            'Radial', 'Ramp', 'Ranch', 'Rapid', 'Rest', 'Ridge', 'River', 'Road', 'Route', 'Row', 'Rue', 'Run', 'Shoal', \
            'Shore', 'Skyway', 'Spring', 'Spur', 'Square', 'Station', 'Stravenue', 'Stream', 'Street', 'Summit', 'Terrace', \
            'Throughway', 'Trace', 'Track', 'Trafficway', 'Trail', 'Trailer', 'Tunnel', 'Turnpike', 'Underpass', 'Union', 'Valley', \
            'Viaduct', 'View', 'Village', 'Ville', 'Vista', 'Walk', 'Wall', 'Way', 'Well', 'Wells', 'Via', 'Spur 408'] 


mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Av": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "road": "Road",
            "Blvd": "Boulevard",
            "blvd": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Expy": "Expressway",
            "LN": "Lane",
            "Ln": "Lane",
            "Pkwy": "Parkway",
            "Webb Chapel Rd 200": "Webb Chapel Road",
            "I-30": "Interstate 30",
            "I 30": "Interstate 30",
            "Blvd E": "Boulevard E",
            "West Main Street #B": "West Main Street",
            "7815 McCallum Blvd 14203 Dallas TX 75252 Kjo": "McCallum Boulevard",
            "I-20": "Interstate 20",
            "I 20": "Interstate 20",
            "Forest Central Drive, Suite 300": "Forest Central Drive",
            "W Illinois Ave #306": "West Illinois Avenue",
            "North Market Street #102": "North Market Street",
            "75062": "",
            "North Saint Paul Street, Suite 2010": "North Saint Paul Street",
            "Canton Street, Suite 202": "Canton Street",
            "Las Colinas Blvd E": "Las Colinas Boulevard East",
            "Reunion Blvd E": "Reunion Boulevard East",
            "North Pearl Street, Suite 1150": "North Pearl Street"

            }
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
    


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

if __name__ == '__main__':
    streets = audit('dallas.osm')
    pprint.pprint(streets)

### update_street_name.py

In [ ]:
"""
Created on Sun Apr 22 17:11:19 2018

@author: Kris
"""

import re

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#https://en.wikipedia.org/wiki/Texas_State_Highway_Spur_408
#https://en.wikipedia.org/wiki/Street_suffix
#https://pe.usps.com/text/pub28/28apc_002.htm
expected = ['Alley', 'Annex', 'Arcade', 'Avenue', 'Bayou', 'Beach', 'Bend', 'Bluff', 'Bottom', 'Boulevard', 'Branch', 'Bridge',\
            'Brook', 'Burg', 'Bypass', 'Camp', 'Canyon', 'Cape', 'Causeway', 'Center', 'Circle', 'Cliff', 'Club', 'Common',\
            'Corner', 'Course', 'Court', 'Cove', 'Creek', 'Crescent', 'Crest', 'Crossing', 'Crossroad', 'Curve', 'Dale', \
            'Dam', 'Divide', 'Drive', 'Estate', 'Expressway', 'Extension', 'Fall', 'Ferry', 'Field', 'Flat', 'Ford', \
            'Forest', 'Forge', 'Fork', 'Fort', 'Freeway', 'Garden', 'Gateway', 'Glen', 'Green', 'Grove', 'Harbor', \
            'Haven', 'Heights', 'Highway', 'Hill', 'Hollow', 'Inlet', 'Island', 'Isle', 'Junction', 'Key', 'Knoll', \
            'Lake', 'Land', 'Landing', 'Lane', 'Light', 'Loaf', 'Lock', 'Lodge', 'Loop', 'Mall', 'Manor', 'Meadow', \
            'Mews', 'Mill', 'Mission', 'Motorway', 'Mount', 'Mountain', 'Neck', 'Orchard', 'Oval', 'Overpass', 'Park', \
            'Parkway', 'Pass', 'Passage', 'Path', 'Pike', 'Pine', 'Place', 'Plain', 'Plaza', 'Point', 'Port', 'Prairie', \
            'Radial', 'Ramp', 'Ranch', 'Rapid', 'Rest', 'Ridge', 'River', 'Road', 'Route', 'Row', 'Rue', 'Run', 'Shoal', \
            'Shore', 'Skyway', 'Spring', 'Spur', 'Square', 'Station', 'Stravenue', 'Stream', 'Street', 'Summit', 'Terrace', \
            'Throughway', 'Trace', 'Track', 'Trafficway', 'Trail', 'Trailer', 'Tunnel', 'Turnpike', 'Underpass', 'Union', 'Valley', \
            'Viaduct', 'View', 'Village', 'Ville', 'Vista', 'Walk', 'Wall', 'Way', 'Well', 'Wells', 'Via', 'Spur 408', 'Central', \
            'Downs'] 

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Av": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "road": "Road",
            "Blvd": "Boulevard",
            "blvd": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Expy": "Expressway",
            "LN": "Lane",
            "Ln": "Lane",
            "Pkwy": "Parkway",
            "Webb Chapel Rd 200": "Webb Chapel Road",
            "I-30": "Interstate 30",
            "I 30": "Interstate 30",
            "Blvd E": "Boulevard E",
            "West Main Street #B": "West Main Street",
            "7815 McCallum Blvd 14203 Dallas TX 75252 Kjo": "McCallum Boulevard",
            "I-20": "Interstate 20",
            "Forest Central Drive, Suite 300": "Forest Central Drive",
            "W Illinois Ave #306": "West Illinois Avenue",
            "North Market Street #102": "North Market Street",
            "75062": "",
            "North Saint Paul Street, Suite 2010": "North Saint Paul Street",
            "Canton Street, Suite 202": "Canton Street",
            "Las Colinas Blvd E": "Las Colinas Boulevard East",
            "Reunion Blvd E": "Reunion Boulevard East",
            "North Pearl Street, Suite 1150": "North Pearl Street",
            "N Central Expressway Ste 635": "N Central Expressway",
            "South Parkway Boulevard South": "South Parkway Boulevard",
            "E State Highway 356": "East State Highway 356",
            "56th": "56th Street",
            "Nile": "Nile Drive",
            "Haskell": "Haskell Drive",
            "Birchbrook": "Birchbrook Drive",
            "North San Saba": "North San Saba Drive",
            "South San Saba": "South San Saba Drive",
            "Cardiff": "Cardiff Street",
            "Barcelona": "Barcelona Drive",
            "Jo Pierce": "Jo Pierce Street",
            "Glenwood": "Glenwood Avenue",
            "Gillette": "Gillette Street",
            "Pleasant Mound": "South Buckner Boulevard",
            "Sheree": "Sheree Lane",
            "Wren": "Wren Way",
            "Wandt": "Wandt Drive",
            "Inwood": "Inwood Road",
            "Vista Gate": "Vista Gate Drive",
            "E Kearney": "East Kearney Street"
          
            }

def update_name(name):
    m = street_type_re.search(name)
    street_type = m.group()

     if (street_type not in expected) & (street_type in mapping):
         #https://lzone.de/examples/Python%20re.sub
         name = re.sub(street_type, mapping[street_type], name)
         

    return name
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"


if __name__ == '__main__':
    test()

### key_type.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 16 18:43:42 2018

@author: Kris
"""

import xml.etree.cElementTree as ET
import pprint

"""
Your task is to explore the data a bit more.
The first task is a fun one - find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")
"""

def get_keys(element):
    return 


def process_map(filename):
    keys = set()
    for _, element in ET.iterparse(filename):
        if 'k' in element.attrib:
            keys.add(element.get('k'))
   

    return keys

def test():
    keys = process_map("dallas.osm")
    pprint.pprint(keys)
    
if __name__ == "__main__":
    test()
    

### data.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus
import schema

from update_street_name import update_name


OSM_PATH = "dallas.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    nodepos = 0 
    
    # YOUR CODE HERE
   
    
    if element.tag == 'node':
        for field in NODE_FIELDS:
            node_attribs[field] = element.attrib[field]
        for tag in element.iter('tag'):
            t_dic = {}
            t_dic['id'] = element.attrib['id'] 
            
            
            if PROBLEMCHARS.match(tag.attrib["k"]):
                continue
                       
            #split at colon into two separate values, like addr:street. Street becomes the key, addr becomes the type.
            #If there is no colon set type to regular.
            elif LOWER_COLON.match(tag.attrib['k']):
                
                t_dic['type'] = tag.attrib['k'].split(':')[0]
                t_dic['key'] = tag.attrib["k"].split(':',1)[1]
                #correct all tags that associated with street.
             
                if tag.attrib['k'] == 'addr:street':
             
                    t_dic['value'] = update_name(tag.attrib['v'])
 
                else:
                    t_dic['value'] = tag.attrib['v']
                
            else:
                t_dic['type'] = 'regular'
                t_dic['key'] = tag.attrib['k']
                t_dic['value'] = tag.attrib['v']
            tags.append(t_dic)



            
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for field in WAY_FIELDS:
            way_attribs[field] = element.attrib[field]
#        Here we have to process the information slightly differently.
#        Each id can have multiple entries so we keep them together and write them according to the id and position.
#        We start at position 0, resetting the position when we encounter a new id.
       
        for nd in element.iter('nd'):
            n_dic = {}
            n_dic['id'] = element.attrib['id']
            n_dic['node_id'] = nd.attrib['ref']
            n_dic['position'] = nodepos
            nodepos += 1
            way_nodes.append(n_dic)
        #repeat the same process for way as we did with node.
        for tag in element.iter('tag'):
            t_dic = {}
            t_dic['id'] = element.attrib['id'] 
            if PROBLEMCHARS.match(tag.attrib["k"]):
                continue
            elif LOWER_COLON.match(tag.attrib['k']):
                t_dic['type'] = tag.attrib['k'].split(':')[0]
                t_dic['key'] = tag.attrib["k"].split(':',1)[1]
                if tag.attrib['k'] == 'addr:street':
                        t_dic['value'] = update_name(tag.attrib['v'])

                else:
                    t_dic['value'] = tag.attrib['v']

            else:
                t_dic['type'] = 'regular'
                t_dic['key'] = tag.attrib['k']
                t_dic['value'] = tag.attrib['v']
            tags.append(t_dic)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()




def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, codecs.open(WAYS_PATH, 'w') as ways_file, codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            
            if el:

                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)


### remove_blank.py

In [ ]:
files = ['nodes.csv','ways.csv', 'ways_tags.csv', 'nodes_tags.csv', 'ways_nodes.csv']
    
for filename in files:
    with open('{}_{}'.format('new',filename), 'w') as f_out:
        for line in open(filename):
            line = line.rstrip()
            if line != '':
                line = line + '\n'
                f_out.write(line)

### db_create.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 24 17:11:15 2018

@author: elikr
"""
# this script creates the sqlite3 database, creates the tables in the db and imports the information from each of the csvs created in the xml to csv conversion process in data.py.
import csv, sqlite3
sql_file="dallas_metro.db"
  
con = sqlite3.connect(sql_file)
cur = con.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes; ''')
con.commit()

cur.execute("CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()

with open('nodes.csv','rb') as f: 
   
    data = csv.DictReader(f) 
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in data]
   

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db,)
con.commit()

cur.execute('''DROP TABLE IF EXISTS ways; ''')
con.commit()

cur.execute("CREATE TABLE ways (id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()

with open('ways.csv','rb') as f:
    
    data = csv.DictReader(f) 
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in data]
    

cur.executemany("INSERT INTO ways (id,user,uid,version,changeset,timestamp) VALUES (?,?,?,?,?,?);", to_db,)
con.commit()

cur.execute('''DROP TABLE IF EXISTS nodes_tags; ''')
con.commit()

cur.execute("CREATE TABLE nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id));") # use your column names here
con.commit()

with open('nodes_tags.csv','rb') as f: 

    data = csv.DictReader(f) 
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in data]
    

cur.executemany("INSERT INTO nodes_tags (id,key,value,type) VALUES (?,?,?,?);", to_db,)
con.commit()


cur.execute('''DROP TABLE IF EXISTS ways_tags; ''')
con.commit()

cur.execute("CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,type TEXT,FOREIGN KEY (id) REFERENCES ways(id));") # use your column names here
con.commit()

with open('ways_tags.csv','rb') as f: 
    data = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in data]
    

cur.executemany("INSERT INTO ways_tags (id,key,value,type) VALUES (?,?,?,?);", to_db,)
con.commit()


cur.execute('''DROP TABLE IF EXISTS ways_nodes; ''')
con.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL, position INTEGER NOT NULL,FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));") 
con.commit()

with open('ways_nodes.csv','rb') as f: 
    data = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in data]
    

cur.executemany("INSERT INTO ways_nodes (id,node_id, position) VALUES (?,?,?);", to_db,)
con.commit()
con.close()
